# Machine Vision - Assignment 8: ViT Implementation

This exercise will follow assignment 7 in structure. However, its goal is to get used to implement state-of-the-art architectures based on reading a paper.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import torchvision
from torchvision.transforms import v2

##### Preparation

##### German Traffic Sign Recognition Benchmark

The German Traffic Sign Recognition Benchmark [(GTSRB)](https://benchmark.ini.rub.de/) is a competition that was held at the IJCNN 2011. In this competition images of traffic signs should be classified.
You will implement your own neural network to classify a subset of the GTSRB dataset. This subset consists of `12` different classes, which are shown in the figures below. However, you are free to extend your solution to the full dataset.


|---|------------------------------|-------------------------------|-------------------------------|-------------------------------|-------------------------------|-------------------------------|-------------------------------|-------------------------------|-------------------------------|--------------------------------|--------------------------------|--------------------------------|
|  ![Class 0](res/images/0.png) | ![Class 1](res/images/6.png) | ![Class 2](res/images/16.png) | ![Class 3](res/images/17.jpg) | ![Class 4](res/images/19.png) | ![Class 5](res/images/22.jpg) | ![Class 6](res/images/28.png) | ![Class 7](res/images/29.png) | ![Class 8](res/images/32.png) | ![Class 9](res/images/33.png) | ![Class 10](res/images/38.png) | ![Class 11](res/images/40.png) |
<br></br>

In order to simplify this exercise, the raw GTSRB images are already transformed into a dataset, where each image has the shape of `[C,H,W]` (Height x Width x Channels) with values ranging from `0-1`.
Furthermore, the dataset is split into a train-, validation- and test-dataset, where the train- and validation-datasets are provided.

In [ ]:
NUM_CLASSES = 12

train_ds = torchvision.datasets.ImageFolder("data_train", transform=v2.Compose([
    v2.RandAugment(),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
]))

val_ds = torchvision.datasets.ImageFolder("data_val", transform=v2.Compose([
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
]))

train_dl = DataLoader(dataset=train_ds, batch_size=8, shuffle=True)
val_dl = DataLoader(dataset=val_ds, batch_size=8, shuffle=False)

Which means, that each label is a vector of 12 entries, where only the entry of the class has the value $1$ and all others values are $0$The `torchvision.datasets.ImageFolder` is a simple way to represent classification datasets. For more information you can read it up here: [ImageFolder](https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html)

Furthermore, the standard pytorch dataloader class is used to create an iterator based on an `torch.utils.data.Dataset` class.
Each iteration, the dataloader returns a batch of `x = [Bx3x32x32]` images and `y = [Bx12]` class labels, where B is the batch size.

There are different approaches to encode the class label. For further information you can read this blog entry [integer- or one-hot-encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/).
In this exercise the labels are encoded in the integer format.

In [ ]:
x_batch, y_batch = next(iter(train_dl))

# @student print the image and label shape of a batch
print(f"image shape: {x_batch.shape}")
print(f"label shape: {y_batch.shape}")

In [ ]:
# @student show one image of the batch and its label
print(f"label: {y_batch[1]}")

plt.imshow(x_batch[0].permute(1,2,0))
plt.show()

### AN IMAGE IS WORTH 16X16 WORDS: TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE

We will follow the steps of the ground braking paper "AN IMAGE IS WORTH 16X16 WORDS: TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE" [Link to Publication](https://arxiv.org/abs/2010.11929)
which generalized the idea of language transformer models to vision tasks.

![vit_architecture](res/images/vit.png)


### Method

Before starting to implement the ViT please read the publication and shortly describe each of these steps:
<br />

##### Step 1: Transform Images into Patches

##### Step 2: Linear embed Patches into Tokens (Embeddings)

##### Step 3: Add Position embedding to Tokens (Embeddings)

##### Step 4: Feed Tokens into multiple Transformer Encoder Layer

##### Step 5: Pool the Result and feed into Classification Head

### Step 1

In [ ]:
# hint: you can easily implement the patching of an image using einops. https://einops.rocks/1-einops-basics/
# However, it is also possible using numpy / for loops
from einops import rearrange
def transform_batch_into_patches(image_batch, patch_size):
    # @student: implement
    ...

print(x_batch.shape)
batch_patches = transform_batch_into_patches(x_batch, patch_size=4)
print(batch_patches.shape)

Make sure tokens are extracted correctly

In [ ]:
img0_patches = batch_patches[0]

rows = 8
cols = 8
fig, axes = plt.subplots(rows, cols, figsize=(4, 4))
axes = axes.flatten()

for i, patch in enumerate(img0_patches):
    # @ student: reshape the flattened patch back to an image
    patch_ = patch.reshape(4,4,3)
    axes[i].imshow(patch_)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

### Step 2, 3, 4, 5

The embedding from patches to tokens is a part of our model, hence we will start with the implementation of our model in this step.
To ease the implementation, an already working Transformer class is available in `transformer.py`.
Please use it until you can train your model to convergence.

In [ ]:
from transformer import Transformer

class VisionTransformer(nn.Module):
    def __init__(self, *,
                 image_size, patch_size,
                 transformer_dim, transformer_depth, transformer_heads=4, transformer_dim_head=32
     ):
        super().__init__()
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'
        self.image_size = image_size
        self.patch_size = patch_size

        # @student: Layers for step 2

        # @student: Layers for step 3
        # Hint: Use the learned position embedding (using a nn.Parameter)

        # @student: Layers for step 4

        # @student: Layers for step 5

    def forward(self, image_batch):
        # @student: (step 1) Transform Images into Patches

        # @student: (step 2) Linear embed Patches into Tokens (Embeddings)

        # @student: (step 3) Add Position embedding to Tokens (Embeddings)

        # @student: (step 4) Feed Tokens (Embeddings) into multiple Transformer Encoder Layer
        # hint: instead of a cls token you can use the mean over the tokens as final embedding per image (Simple ViT) https://arxiv.org/abs/2205.01580

        # @student: (step 5)
        return ...


In [ ]:
model = VisionTransformer(
    image_size=32, patch_size=4,
    transformer_dim=48, transformer_depth=6, transformer_heads=8, transformer_dim_head=16
)
print(model)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

def eval(model, dl, with_cm=False):
    y_labels = []
    pred_labels = []
    model.eval()
    with torch.no_grad():
        for i, (imgs, labels) in enumerate(dl):
            logits = model(imgs)

            # @student: calculate class predictions based on the logits
            preds = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)

            pred_labels.extend(preds.detach().numpy())
            y_labels.extend(labels.detach().numpy())

    if with_cm:
        cm = confusion_matrix(y_labels, pred_labels)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.show()
    model.train()
    print(f"f1: {f1_score(y_labels, pred_labels, average='macro')}, p: {precision_score(y_labels, pred_labels, average='macro')}, r: {recall_score(y_labels, pred_labels, average='macro')}")

Below is the training process of the model.
If your machine supports gpu acceleration uncomment line 6.

In [ ]:
for epoch in range(10):
    print(f"epoch {epoch+1}")
    pbar = tqdm(enumerate(train_dl))
    running_loss = []
    for i, (imgs, labels) in pbar:
        # @student: uncomment if your machine has GPU support
        #imgs, labels = imgs.to("cuda"), labels.to("cuda")

        optimizer.zero_grad()

        preds = model(imgs)
        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        running_loss.append(loss.detach().numpy())
        pbar.set_description(f"loss {np.mean(running_loss):.3} - " )
        pbar.update()
    eval(model, val_dl)


#### From Logits to Labels

Your network will output logits and not the final predictions.
Hence, you further need to calculate the predicted label based on the logits.

In [ ]:
eval(model, val_dl, with_cm=True)

Once you feel confident with your model evaluate it one last time on the test set.
The test set will be uploaded during the exercise session.

In [ ]:
# @student: final check of your fully trained model
test_ds = torchvision.datasets.ImageFolder("data_test", transform=v2.Compose([
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
]))
test_dl = DataLoader(dataset=test_ds, batch_size=16, shuffle=False)

eval(model, test_dl)